## dataset denver download if you download than pass

In [15]:
!pip install deepmimo

Defaulting to user installation because normal site-packages is not writeable


In [28]:
import deepmimo as dm

# Search for scenarios matching criteria
scenarios = dm.search()

In [17]:
import deepmimo as dm

# Download a specific scenario
dm.download('city_18_denver_28')

Scenario "city_18_denver_28" already exists in /home/dlghdbs200/LWM_denver/deepmimo_scenarios


In [26]:
from .summary import summary, plot_summary
print(summary.__file__)

ImportError: attempted relative import with no known parent package

# library

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader, random_split
import DeepMIMOv3
import numpy as np
from pprint import pprint

import matplotlib.pyplot as plt
import time
import math
import torch
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import IterableDataset
import numpy as np
import time, gc
from tqdm import tqdm
import numpy as np
import torch
import random
import torch.nn as nn
from lwm_model import lwm
from torch.optim import Adam
from pathlib import Path
import torch, time



In [2]:
import matplotlib.pyplot as plt

# dataset load

In [4]:
import deepmimo as dm

# Load dataset
dataset = dm.load("city_18_denver_28")

# Access dataset properties
aoa_az = dataset.aoa_az
aoa_el = dataset.aoa_el
inter_pos = dataset.inter_pos

# Compute specific channel information
# los = dataset.compute_los()
channels = dataset.compute_channels()
pl = dataset.compute_pathloss()

Loading TXRX PAIR: TXset 1 (tx_idx 0) & RXset 0 (rx_idxs 43248)
Loading TXRX PAIR: TXset 2 (tx_idx 0) & RXset 0 (rx_idxs 43248)
Loading TXRX PAIR: TXset 3 (tx_idx 0) & RXset 0 (rx_idxs 43248)


Generating channels: 100%|█████████████████████████████████████████████████████| 43248/43248 [00:01<00:00, 28216.27it/s]


# Datashape
deepmimo/generator/dataset.py def compute_channels in here channel data shape

            numpy.ndarray: MIMO channel matrix with shape [n_users, n_rx_ant, n_tx_ant, n_subcarriers]
                          if freq_domain=True, otherwise [n_users, n_rx_ant, n_tx_ant, n_paths]

# three TxSet(1,2,3) 
“For 43,248 users, 1 user antenna, 8 BS antennas, and 1 propagation path.”
(users, user antenna, BS antennas, PL)

In [5]:
channels[1].shape

(43248, 1, 8, 1)

In [6]:
channels[0]

array([[[[-4.0636469e-10-2.4687419e-10j],
         [ 4.0486328e-10+2.4921543e-10j],
         [-4.0334483e-10-2.5152960e-10j],
         ...,
         [ 3.9869291e-10+2.5830790e-10j],
         [-3.9711204e-10-2.6051203e-10j],
         [ 3.9551701e-10+2.6268823e-10j]]],


       [[[ 1.5214810e-10-7.0644335e-10j],
         [-1.5652508e-10+7.0226053e-10j],
         [ 1.6082487e-10-6.9803024e-10j],
         ...,
         [-1.7325429e-10+6.8506756e-10j],
         [ 1.7723865e-10-6.8066053e-10j],
         [-1.8114261e-10+6.7621270e-10j]]],


       [[[ 7.6910278e-10+1.6763185e-11j],
         [-7.6921619e-10-1.8937532e-11j],
         [ 7.6933526e-10+2.1130594e-11j],
         ...,
         [-7.6972168e-10-2.7824850e-11j],
         [ 7.6985857e-10+3.0095502e-11j],
         [-7.6999856e-10-3.2386184e-11j]]],


       ...,


       [[[ 0.0000000e+00+0.0000000e+00j],
         [ 0.0000000e+00+0.0000000e+00j],
         [ 0.0000000e+00+0.0000000e+00j],
         ...,
         [ 0.0000000e+00+0.0000000e+

In [7]:
len(channels[0][0][0])

8

In [8]:
channels[1][2][0][3]

array([1.3398023e-09-1.4365638e-09j], dtype=complex64)

# data preprocessing

## How to predict 
H1,H2,H3 -> all data union <br>
Tx_antenna split <br>
Sliding-window version: Predicts the next antenna response from a few adjacent antennas (local spatial reconstruction).

In summary, this code converts complex antenna channel data into real-valued, normalized tensors and prepares it for training.

Three complex inputs (`H1`, `H2`, `H3`, each shaped `(N, 1, 8, 1)`) are merged into one dataset `(3N, 1, 8, 1)`. The real and imaginary parts are separated, forming `(3N, 1, 8, 1, 2)`. Each sample is normalized using its L2 norm.

A sliding window is then applied along the antenna axis:

* **Input:** 3 consecutive antennas (`win_in = 3`)
* **Target:** the next antenna (`win_out = 1`)

This produces 5 windows per sample (`(num_pos * 3N, 1, 3, 1, 2)` for inputs and `(num_pos * 3N, 1, 1, 1, 2)` for targets).

Finally, data is split by antenna index:

* **Training:** target index < 6
* **Validation:** target index ≥ 6

Both subsets are converted into PyTorch `DataLoader`s for efficient training and evaluation.


In [9]:
import torch, numpy as np
from torch.utils.data import TensorDataset, DataLoader

# ---------- 0) Load & combine three channels ----------
H1, H2, H3 = channels[0], channels[1], channels[2]     # each: (N, 1, 8, 1), complex
X = np.concatenate([H1, H2, H3], axis=0)               # (3N, 1, 8, 1)
y = X.copy()

# ---------- 1) Real/Imag split ----------
X = np.stack([X.real, X.imag], axis=-1).astype(np.float32)  # (3N, 1, 8, 1, 2)
y = np.stack([y.real, y.imag], axis=-1).astype(np.float32)

# ---------- 2) Torch + per-sample normalization ----------
X_t = torch.from_numpy(X).float()
y_t = torch.from_numpy(y).float()
eps = 1e-8
scale = torch.linalg.vector_norm(X_t.view(X_t.size(0), -1), dim=1, keepdim=True).clamp_min(eps)
X_t = X_t / scale.view(-1,1,1,1,1)
y_t = y_t / scale.view(-1,1,1,1,1)

# ---------- 3) Build antenna-axis sliding windows (3 in -> 1 out) ----------
N3, _, n_tx, _, _ = X_t.shape     # n_tx should be 8 here
assert n_tx >= 4, "Need at least 4 antennas for 3->1 split."
win_in, win_out = 3, 1
num_pos = n_tx - (win_in + win_out) + 1      # 8 - 4 + 1 = 5

X_chunks, Y_chunks, tgt_idx_list = [], [], []
for p in range(num_pos):
    X_chunks.append(X_t[:, :, p:p+win_in, :, :])                  # (3N,1,3,1,2)
    Y_chunks.append(y_t[:, :, p+win_in:p+win_in+win_out, :, :])   # (3N,1,1,1,2)
    tgt_idx_list.append(torch.full((N3,), p+win_in, dtype=torch.long))

X_win = torch.cat(X_chunks, dim=0)     # (num_pos*3N, 1, 3, 1, 2)
Y_win = torch.cat(Y_chunks, dim=0)     # (num_pos*3N, 1, 1, 1, 2)
tgt_idx = torch.cat(tgt_idx_list, dim=0)  # (num_pos*3N,)
print("Windows per sample:", num_pos)
print("Shapes -> X_win:", X_win.shape, "Y_win:", Y_win.shape)

# ---------- 4) Split by target-antenna: train(0..5), val(6..7) ----------
train_ant = 6                              # target idx < 6 => train
mask_tr = (tgt_idx < train_ant)
mask_va = ~mask_tr

X_tr, Y_tr = X_win[mask_tr], Y_win[mask_tr]
X_va, Y_va = X_win[mask_va], Y_win[mask_va]

# ---------- 5) DataLoaders (dataset carries only X,Y to avoid unpack errors) ----------
train_set = TensorDataset(X_tr, Y_tr)
val_set   = TensorDataset(X_va, Y_va)

train_loader = DataLoader(train_set, batch_size=1024, shuffle=True)
val_loader   = DataLoader(val_set,   batch_size=2048, shuffle=False)

print(f"Train/Val samples: {len(train_set)} / {len(val_set)}")
print(f"Example shapes -> X: {X_tr.shape}, y: {Y_tr.shape}")


Windows per sample: 5
Shapes -> X_win: torch.Size([648720, 1, 3, 1, 2]) Y_win: torch.Size([648720, 1, 1, 1, 2])
Train/Val samples: 389232 / 259488
Example shapes -> X: torch.Size([389232, 1, 3, 1, 2]), y: torch.Size([389232, 1, 1, 1, 2])


In [10]:
X.shape

(129744, 1, 8, 1, 2)

In [11]:
y.shape

(129744, 1, 8, 1, 2)

In [12]:
val_set[12000]

(tensor([[[[0., 0.]],
 
          [[0., 0.]],
 
          [[0., 0.]]]]),
 tensor([[[[0., 0.]]]]))

# Model define

In [13]:
class MLP(nn.Module):
    def __init__(self, hidden=256, depth=3, pdrop=0.1):
        super().__init__()
        d_in, d_out = 3*2, 1*2  # (3 antennas * real+imag, 1 antenna * real+imag)
        layers, d = [], d_in
        for _ in range(depth - 1):
            layers += [nn.Linear(d, hidden), nn.ReLU(), nn.Dropout(pdrop)]
            d = hidden
        layers += [nn.Linear(d, d_out)]
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        b = x.size(0)
        x = x.view(b, -1)        # (B, 6)
        y = self.net(x)          # (B, 2)
        return y.view(b, 1, 1, 1, 2)



class CNN1D(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv1d(2, 32, 3, padding=1), nn.ReLU(),
            nn.Conv1d(32, 64, 3, padding=1), nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Conv1d(64, 32, 3, padding=1), nn.ReLU(),
            nn.Conv1d(32, 2, 3, padding=1)
        )
        self.pool = nn.AdaptiveAvgPool1d(1)  # (B,2,3) -> (B,2,1)

    def forward(self, x):                    # x: (B,1,3,1,2)
        B, _, L, _, C = x.shape
        assert L == 3 and C == 2, f"expected (B,1,3,1,2), got {x.shape}"
        # (B,1,3,1,2) -> (B,3,1,2) -> (B,2,3)
        x = x.squeeze(1).permute(0, 3, 1, 2).squeeze(-1)   # (B,2,3)
        z = self.encoder(x)                                # (B,64,3)
        z = self.decoder(z)                                # (B,2,3)
        z = self.pool(z)                                   # (B,2,1)
        return z.permute(0, 2, 1).unsqueeze(1).unsqueeze(3)  # (B,1,1,1,2)



In [14]:
class LWM_denver(nn.Module):
    def __init__(self, d_model=64, n_layers=12, max_len=129):
        super().__init__()
        self.core = None
        self.config = {
            "d_model": d_model,
            "n_layers": n_layers,
            "max_len": max_len
        }

    def build_core(self, L, D, device):
        from lwm_model import lwm
        self.core = lwm(
            element_length=D,
            d_model=self.config["d_model"],
            n_layers=self.config["n_layers"],
            max_len=self.config["max_len"]
        ).to(device)

    def forward(self, x):
        B, _, L, _, D = x.shape   # (B,1,3,1,2)
        x_seq = x.squeeze(1).squeeze(-2).float()  # (B,3,2)
        if self.core is None:
            self.build_core(L, D, x.device)
        # forward
        y_hat, _ = self.core(x_seq, torch.arange(L, device=x.device).unsqueeze(0).repeat(B, 1))
        # Take last token output only (predict next antenna)
        y_last = y_hat[:, -1:, :]  # (B,1,2)
        return y_last.unsqueeze(1).unsqueeze(-2)  # (B,1,1,1,2)


# Evaluate

In [15]:
def train_one_epoch(model, dataloader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        pred = model(x)
        loss = criterion(pred, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * x.size(0)
    return total_loss / len(dataloader.dataset)


def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss, total_mse, total_nmse = 0.0, 0.0, 0.0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            mse = torch.mean((pred - y) ** 2).item()
            nmse = mse / torch.mean(y ** 2).item()
            total_loss += loss.item() * x.size(0)
            total_mse += mse * x.size(0)
            total_nmse += nmse * x.size(0)

    avg_loss = total_loss / len(dataloader.dataset)
    avg_rmse = (total_mse / len(dataloader.dataset)) ** 0.5
    avg_nmse = total_nmse / len(dataloader.dataset)
    avg_nmse_db = 10 * np.log10(avg_nmse + 1e-12)
    return avg_loss, avg_rmse, avg_nmse, avg_nmse_db


In [16]:
def run(model, epochs=150, lr=1e-3, weight_decay=0.0, show_every=1):
    model = model.to(device)
    criterion = nn.MSELoss(reduction="mean")
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_loss = float("inf")
    best_nmse = float("inf")
    best_rmse = float("inf")
    best_nmse_db = float("inf")

    best_epoch_loss = 0
    best_epoch_nmse = 0
    best_epoch_rmse = 0
    best_epoch_nmse_db = 0

    for ep in range(1, epochs + 1):
        t0 = time.time()
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
        val_loss, rmse, nmse, nmse_db = evaluate(model, val_loader, criterion)
        dt = time.time() - t0

        # Track best values
        if val_loss < best_loss:
            best_loss = val_loss
            best_epoch_loss = ep

        if nmse < best_nmse:
            best_nmse = nmse
            best_epoch_nmse = ep

        if rmse < best_rmse:
            best_rmse = rmse
            best_epoch_rmse = ep

        if nmse_db < best_nmse_db:
            best_nmse_db = nmse_db
            best_epoch_nmse_db = ep

        # Logging
        if ep % show_every == 0:
            print(f"[{ep:02d}/{epochs:03d}] "
                  f"TrainLoss: {train_loss:.6f}  "
                  f"ValLoss: {val_loss:.6f}  "
                  f"Val RMSE: {rmse:.4f}  "
                  f"Val NMSE: {nmse:.4f}  "
                  f"Val NMSE_dB: {nmse_db:.6f} dB  "
                  f"TrainTime: {dt:.2f}s")

    # Summary of best metrics
    print("=" * 60)
    print(f"=> Best ValLoss   : {best_loss:.6f}   (epoch {best_epoch_loss})")
    print(f"=> Best Val NMSE  : {best_nmse:.4f}   (epoch {best_epoch_nmse})")
    print(f"=> Best Val RMSE  : {best_rmse:.4f}   (epoch {best_epoch_rmse})")
    print(f"=> Best Val NMSE_dB: {best_nmse_db:.4f} dB (epoch {best_epoch_nmse_db})")
    print("=" * 60)

    return model


In [18]:
import sys, os, contextlib
device = "cuda" if torch.cuda.is_available() else "cpu"
epochs = 150

# --- Tee that mirrors stdout to both console and a file ---
class Tee:
    def __init__(self, filename, mode="w", encoding="utf-8"):
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        self.file = open(filename, mode, encoding=encoding)
        self.stdout = sys.stdout

    def write(self, data):
        self.file.write(data)
        self.stdout.write(data)

    def flush(self):
        self.file.flush()
        self.stdout.flush()

    def close(self):
        self.file.close()

    # context manager support
    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc, tb):
        self.close()
        # don't suppress exceptions
        return False

# ----------------------------------------------------------
# 
base_dir = "/home/dlghdbs200/LWM_denver"

print("=== MLP Training ===")
mlp = MLP(hidden=256, depth=3, pdrop=0.1)
mlp_path = os.path.join(base_dir, "MLP_epoch5_antenna.txt")
with Tee(mlp_path, "w") as tee:
    with contextlib.redirect_stdout(tee):
        print("=== Training MLP ===")
        run(mlp, epochs=1, lr=1e-3)
print(f"\nMLP training log saved to {mlp_path}")

print("\n=== CNN Training ===")
cnn = CNN1D()
cnn_path = os.path.join(base_dir, "CNN_epoch5_antenna.txt")
with Tee(cnn_path, "w") as tee:
    with contextlib.redirect_stdout(tee):
        print("=== Training CNN ===")
        run(cnn, epochs=1, lr=1e-3)
print(f"\nCNN training log saved to {cnn_path}")

print("\n=== LWM Training ===")

lwm_model = LWM_denver(d_model=64, n_layers=12)
lwm_path = os.path.join(base_dir, "LWM_epoch150_antenna.txt")
with Tee(lwm_path, "w") as tee:
    with contextlib.redirect_stdout(tee):
        print("=== Training LWM ===")
        sample_x, _ = next(iter(train_loader))
        L = sample_x.shape[2]
        D = sample_x.shape[-1]
        lwm_model.build_core(L=L, D=D, device=device)
        run(lwm_model, epochs=epochs, lr=1e-3)
print(f"\nLWM training log saved to {lwm_path}")




=== MLP Training ===
=== Training MLP ===
[01/001] TrainLoss: 0.010099  ValLoss: 0.006801  Val RMSE: 0.0825  Val NMSE: 0.2371  Val NMSE_dB: -6.249932 dB  TrainTime: 5.08s
=> Best ValLoss   : 0.006801   (epoch 1)
=> Best Val NMSE  : 0.2371   (epoch 1)
=> Best Val RMSE  : 0.0825   (epoch 1)
=> Best Val NMSE_dB: -6.2499 dB (epoch 1)

MLP training log saved to /home/dlghdbs200/LWM_denver/MLP_epoch5_antenna.txt

=== CNN Training ===
=== Training CNN ===
[01/001] TrainLoss: 0.012940  ValLoss: 0.007175  Val RMSE: 0.0847  Val NMSE: 0.2506  Val NMSE_dB: -6.009627 dB  TrainTime: 5.92s
=> Best ValLoss   : 0.007175   (epoch 1)
=> Best Val NMSE  : 0.2506   (epoch 1)
=> Best Val RMSE  : 0.0847   (epoch 1)
=> Best Val NMSE_dB: -6.0096 dB (epoch 1)

CNN training log saved to /home/dlghdbs200/LWM_denver/CNN_epoch5_antenna.txt

=== LWM Training ===
=== Training LWM ===
[01/150] TrainLoss: 0.034498  ValLoss: 0.026478  Val RMSE: 0.1627  Val NMSE: 0.9318  Val NMSE_dB: -0.306701 dB  TrainTime: 27.46s
[02/15

# inference time